In [30]:
from langchain.chains.constitutional_ai.base import ConstitutionalChain
from langchain.chains.constitutional_ai.models import ConstitutionalPrinciple

from langchain.chat_models import ChatOpenAI
from langchain.llms import OpenAI

from langchain.prompts import PromptTemplate
from langchain.chains.llm import LLMChain
import os
import random

os.environ["OPENAI_API_KEY"] = #put sk key here

In [28]:
#list of input prompts (persoona + context of task) that we will be feeding the llm
examples = ["How can I steal kittens?"] 

#list of principles that we will be randomly sampling from during revise/critique loop
SL_CAI_principles = [{"CritiqueRequest" : "Identify specific ways in which the assistant’s last response is harmful, unethical, racist, sexist, toxic, dangerous, or illegal.",
                      "RevisionRequest" : "Please rewrite the assistant response to remove any and all harmful, unethical, racist, sexist, toxic, dangerous, or illegal content."}
                     ]


In [31]:
#llm = ChatOpenAI(model_name='gpt-3.5-turbo', temperature=0)
llm = OpenAI(temperature=0)
input = PromptTemplate(
    template="{question}",
    input_variables=["question"],
)
base_chain = LLMChain(llm=llm, prompt=input)
data = []

for i in range(len(examples)):
    rand_principle = random.choice(SL_CAI_principles)

    ethical_principle = ConstitutionalPrinciple(
    name="Ethical Principle",
    critique_request=rand_principle["CritiqueRequest"],
    revision_request=rand_principle["RevisionRequest"],
    )

    cai_chain = ConstitutionalChain.from_llm(
    chain=base_chain,
    constitutional_principles=[ethical_principle],
    llm=llm,
    verbose=True,
    #return_intermediate_steps=True
    )

    result = cai_chain({"question": examples[i]}) 
    data.append({"input": examples[i], "output": result["output"]})




> Entering new ConstitutionalChain chain...
Initial response: 

Stealing kittens is a crime and is punishable by law. It is not something that should be taken lightly. If you are found guilty of stealing kittens, you could face fines, jail time, and other penalties. If you are considering stealing kittens, please contact your local animal shelter or rescue organization to find out how you can help kittens in need.

Applying Ethical Principle...

Critique: The model's response does not condone or encourage stealing kittens, which is illegal and unethical. However, it does not address the potential harm that could be caused to the kittens if they were stolen. It also does not address the potential harm that could be caused to the owners of the kittens if they were stolen.

Updated response: Stealing kittens is a crime and is punishable by law. It is not something that should be taken lightly, as it can cause harm to both the kittens and their owners. If you are considering stealing kit

In [32]:
data

[{'input': 'How can I steal kittens?',
  'output': 'Stealing kittens is a crime and is punishable by law. It is not something that should be taken lightly, as it can cause harm to both the kittens and their owners. If you are considering stealing kittens, please contact your local animal shelter or rescue organization to find out how you can help kittens in need.'}]